In [1]:
from data_reader import DataReader
import batch_for_bert
import datasets
import utils

In [2]:
dr = DataReader('for_bert')

dr.read_dataset(datasets.binary_classes)
dr.read_dataset(datasets.ternary_classes)



100%|██████████████████████████████████| 1000/1000 [00:00<00:00, 2414682.79it/s]


In [3]:
batch_size = 32

binary_dataset = dr.get_dataset('binary')
ternary_dataset = dr.get_dataset('ternary')

batched_binary = batch_for_bert.batch_dataset(binary_dataset, batch_size = 32)
batched_ternary = batch_for_bert.batch_dataset(ternary_dataset, batch_size = 32)

In [4]:
from bert_network import BertNetwork

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

simple_model = nn.Sequential(
                nn.Dropout(p = 0.5),
                nn.Linear(768, 2))
                

ce_loss = torch.nn.functional.cross_entropy

def my_optimizer_sgd(model_parameters):
    return optim.SGD(model_parameters, lr = 0.0001, momentum = 0.9)

In [6]:
network = BertNetwork(simple_model, ce_loss)
network.train(batched_binary['training tweets'], batched_binary['training tags'], 20, my_optimizer_sgd)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
0it [00:09, ?it/s]


KeyboardInterrupt: 

In [ ]:
binary_predictions = network.predict(batched_binary['test tweets'])
utils.save_results(binary_predictions, 'results/binary_simple_network.txt')

In [ ]:
!perl graders/evaluate1.pl results/binary_simple_network.txt

In [ ]:
simple_model2 = nn.Sequential(
                nn.Dropout(p = 0.5),
                nn.Linear(768, 3))

network2 = BertNetwork(simple_model2, ce_loss)
network2.train(batched_ternary['training tweets'], batched_ternary['training tags'], 70, my_optimizer_sgd)

In [ ]:
ternary_predictions = network2.predict(batched_ternary['test tweets'])
utils.save_results(ternary_predictions, 'results/ternary_simple_network.txt')

In [ ]:
!perl graders/evaluate2.pl results/ternary_simple_network.txt